# Import & Dataset 읽어오기

In [1]:
from datasets import concatenate_datasets, load_from_disk
from src.retriever.retrieval.sparse_retrieval import SparseRetrieval
from transformers import AutoTokenizer

org_dataset = load_from_disk('data/train_dataset')
print(org_dataset)
full_ds = concatenate_datasets(
        [
            org_dataset["train"].flatten_indices(),
            org_dataset["validation"].flatten_indices(),
        ]
    )  # train dev 를 합친 4192 개 질문에 대해 모두 테스트
print("*" * 40, "query dataset", "*" * 40)
print(full_ds)

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 240
    })
})
**************************************** query dataset ****************************************
Dataset({
    features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
    num_rows: 4192
})


# Retriever 선언 및 Retriever 진행
* 아마 초기엔 학습 진행하는데 시간 좀 걸릴예정
    * 약 15분정도 소요(승범 컴 기준)
    * 노션에 mrc/검증결과/Sparse 처리에 가장 좋은 bm25올려났으므로 그거 data 폴더 안에 넣으면 실행가능.

In [2]:
# 위에서 선언한거 가져오기 Retriever
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased", use_fast=False,)
retriever = SparseRetrieval(
    tokenize_fn=tokenizer.tokenize,
    data_path="./data/",
    context_path="wikipedia_documents.json",
    mode = "bm25",
    max_feature=1000000,
    ngram_range=(1,2),
    #tokenized_docs = tokenized_docs,
)
retriever.get_sparse_embedding()

# Top k 조절.
df = retriever.retrieve(full_ds, topk=10)

# 여기서 df가 reader가 사용할 데이터 프레임입니다.
df.head(5)

Lengths of unique contexts : 56737
Building bm25 embedding...
Start Initializing...


Tokenizing...: 100%|██████████| 56737/56737 [02:22<00:00, 399.20it/s]


Generating n-grams and building vocabulary...


Generating n-grams: 100%|██████████| 56737/56737 [00:35<00:00, 1604.14it/s]


Current mode : bm25
End Initialization


Calculating BM25: 100%|██████████| 57/57 [23:33<00:00, 24.80s/it]


Finish BM25 Embedding
bm25 embedding shape: (56737, 1000000)
(4192, 1000000) (56737, 1000000)
<class 'scipy.sparse._csr.csr_matrix'> <class 'scipy.sparse._csr.csr_matrix'>
result shape : (4192, 56737)
[query exhaustive search] done in 58.911 s


Sparse retrieval:   0%|          | 0/4192 [00:00<?, ?it/s]

,question,id,retrieval_context,original_context,answers
0,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,이하의 국가들은 의회에서 행정부 수반을 선출한다는 점에서 내각제와 닮았다. 그리고 ...,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,"{'answer_start': [235], 'text': ['하원']}"
1,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,"{'answer_start': [212], 'text': ['《경영의 실제》']}"
2,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,"1943년 초, 마리아 발토르타가 9년 간 건강 상태가 안 좋아졌을 때, 그녀의 고...",강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,"{'answer_start': [510], 'text': ['백성']}"
3,11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,이 불상군은 1978년 발견되어 학계에 알려졌으며 봉황리 햇골산 중턱 두 곳에 약간...,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...","{'answer_start': [625], 'text': ['중국']}"
4,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,양산 통도사의 금동천문도의 전면에는 천구(天球)의 북극을 중심으로 둥글게 북극으로 ...,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,"{'answer_start': [30], 'text': ['4개']}"


* question : 질문
* id : mrc 번호
* retrieval_context : retrieval 결과로 나온 topk를 하나의 text로 만든것
* original_context : 실제 retrieval 정답
* answers: dict 형태
    * answers_start : original_context에서 어디에서 시작하는지
    * text: 정답의 text값이 뭔지 

In [3]:
len(df['retrieval_context'][0])

13059

## Retriever 결과물 DatasetDict로 저장
reader는 데이터셋을 DatasetDict형태로 받으므로, df에서 변환이 필요하다

In [4]:
from datasets import Dataset, DatasetDict

# df를 Dataset으로
retriever_dataset = Dataset.from_pandas(df)

# DatasetDict로 변환 (train과 validation을 동일 데이터로 초기화)
dataset_dict = DatasetDict({
    'train': retriever_dataset,
    'validation': retriever_dataset
})

# train 스플릿: 'original_context'를 'context'로 사용
dataset_dict['train'] = dataset_dict['train'].remove_columns(['retrieval_context']).rename_column('original_context', 'context')

# validation 스플릿: 'retrieval_context'를 'context'로 사용
dataset_dict['validation'] = dataset_dict['validation'].remove_columns(['original_context']).rename_column('retrieval_context', 'context')

print(dataset_dict)


DatasetDict({
    train: Dataset({
        features: ['question', 'id', 'context', 'answers'],
        num_rows: 4192
    })
    validation: Dataset({
        features: ['question', 'id', 'context', 'answers'],
        num_rows: 4192
    })
})


In [5]:
# 별도 파일로 저장
dataset_dict.save_to_disk('outputs/bm25')

Saving the dataset (0/1 shards):   0%|          | 0/4192 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4192 [00:00<?, ? examples/s]

In [2]:
# csv파일을 읽어오기
from datasets import load_dataset, DatasetDict

# 스플릿을 validation으로 읽기 %csv 하나만 읽으면 'train' 스플릿으로 읽으니 주의
retriever_dataset = load_dataset('csv', data_files='data/test (1).csv')

# 불러온 Dataset 확인
print(retriever_dataset)

# train을 validation으로 변경
dataset_dict = DatasetDict({
    'validation': retriever_dataset['train']
})

# validation 스플릿: 'retrieval_context'를 'context'로 사용
dataset_dict['validation'] = dataset_dict['validation'].remove_columns('Unnamed: 0').rename_column('retrieval_context', 'context')

print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'question', 'id', 'retrieval_context'],
        num_rows: 600
    })
})
DatasetDict({
    validation: Dataset({
        features: ['question', 'id', 'context'],
        num_rows: 600
    })
})


# Reader

In [3]:
from transformers import HfArgumentParser
from transformers import TrainingArguments
from datasets import DatasetDict

from src.reader.model.reader import Reader
from src.utils.arguments import DataTrainingArguments, ModelArguments

#args 로드
model_args = ModelArguments()
data_args =  DataTrainingArguments()
training_args = TrainingArguments(output_dir='outputs')

# 세부 세팅
# model_args.model_name_or_path = <모델경로/huggingface>
model_args.model_name_or_path = 'outputs'


#training_args.do_train =True
#training_args.do_eval = True
training_args.do_predict =True
# 디스크로 데이터셋 불러오기
#dataset = DatasetDict.load_from_disk('data/train_dataset')

#데이터셋 바로 받아올 경우
dataset = dataset_dict


# output_dir은 training_args에 설정된 값을 사용
print("Output directory:", training_args.output_dir)
print(dataset)


reader_model = Reader(model_args, data_args, training_args, dataset)
reader_model.run()




Output directory: outputs
DatasetDict({
    validation: Dataset({
        features: ['question', 'id', 'context'],
        num_rows: 600
    })
})


Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Pre-processing...
['question', 'id', 'context']
Dataset({
    features: ['question', 'id', 'context'],
    num_rows: 600
})
preprocessed Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 14401
})


/opt/conda/lib/python3.10/site-packages/transformers/trainer_pt_utils.py:483: FutureWarning: DistributedTensorGatherer is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


postprocess_qa Dataset({
    features: ['question', 'id', 'context'],
    num_rows: 600
})


  0%|          | 0/600 [00:00<?, ?it/s]

2024-10-24 02:49:43,242 - postprocess log - INFO - Saving predictions to outputs/predictions.json.
2024-10-24 02:49:43,244 - postprocess log - INFO - Saving nbest_preds to outputs/nbest_predictions.json.
2024-10-24 02:49:43,397 - outputs - INFO - Predictions saved to outputs/predictions.json
